<a href="https://colab.research.google.com/github/shreyas-sreedhar/gen-ai-csye7380/blob/main/Assignment-03/Assignment03_1_Shreyas_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSYE7380 - Assignment 03_A - Shreyas Sreedhar

A) Demonstrate LangChain with a suitable example involving RAG, input/output parsing, output formatting, etc.



LLM Used: OpenAI GPT3.5

Embeddings: OpenAI Embeddings



Step by Step-:
Import the Students python notebook as the document,
Let the file be created as embeddings,
Vector Embedd the ipynb file and store locally instead of vectordb to save time and costs.
Use the embedd as context and ask chatgpt 3.5 to answer based on it.
Neatly format the output and display it

In [1]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 576.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succe

In [2]:
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 60.0 MB/s eta 0:00:00


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.7 MB/s eta 0:00:00


In [4]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [7]:
import json

file_path = '/content/Assignment02_Shreyas_S.ipynb'
#this is a run time path, add file everytimee!

with open(file_path, 'r') as file:
    notebook_data = json.load(file)

def extract_notebook_content(notebook_data):
    text_content = []
    for cell in notebook_data.get('cells', []):
        if cell['cell_type'] == 'markdown':
            text_content.append(' '.join(cell['source']))
        elif cell['cell_type'] == 'code':
            text_content.append(' '.join(cell['source']))
    return ' '.join(text_content)

notebook_text = extract_notebook_content(notebook_data)
notebook_text[:500]  # Display first 500 characters


'<a href="https://colab.research.google.com/github/shreyas-sreedhar/gen-ai-csye7380/blob/main/Assignment-02/Assignment02_Shreyas_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> # CSYE7380 - Assignment 02_1 - Shreyas Sreedhar ## 1) Demonstrate the effectiveness of RAG to cite sources and to prevent hallucination. Thought process: Best way to show effectiveness of RAG would be attaching a document with info and comparing it wit'

In [8]:
import os
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from rich import print
from rich.panel import Panel
from rich.text import Text
from dotenv import load_dotenv

In [9]:
file_path = '/content/Assignment02_Shreyas_S.ipynb'

In [10]:

with open(file_path, 'r') as file:
    notebook_data = json.load(file)

def extract_notebook_content(notebook_data):
    text_content = []
    for cell in notebook_data.get('cells', []):
        # Get markdown and code cells content
        if cell['cell_type'] in ['markdown', 'code']:
            text_content.append(' '.join(cell['source']))
    return ' '.join(text_content)


In [11]:

notebook_text = extract_notebook_content(notebook_data)

# Split the notebook text into manageable chunks (e.g., sentences)
texts = notebook_text.split('. ')  #  split based on sentence

### Step 3: Wrap each text chunk in a Document object

In [12]:
documents = [Document(page_content=text) for text in texts]

In [13]:
api_key = OPENAI_API_KEY
embedding_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [14]:
# Step 5: Create a FAISS vector store from the Document objects
vector_store = FAISS.from_documents(documents, embedding_model)

In [15]:
#Step 6: Set up the LangChain RAG pipeline (Retrieval-Augmented Generation)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

### Step 7: Define a user question for querying the notebook content


In [17]:
user_question = "What is the main focus of this notebook?"

In [18]:

# Get the response from the RAG model
response = qa_chain.invoke({"query": user_question})


### Step 8: Formating output

In [19]:

question_text = Text(f"Question: {user_question}", style="bold blue")

# Format the answer with context
answer_text = Text(f"Answer: {response['result']}", style="green")


In [20]:
panel = Panel.fit(
    f"{question_text}\n\n{answer_text}",
    title="The LangChain RAG Output",
    border_style="bold magenta"
)
print(panel)

╭─────────────────────────────────────────── The LangChain RAG Output ────────────────────────────────────────────╮
│ Question: What is the main focus of this notebook?                                                              │
│                                                                                                                 │
│ Answer: The main focus of the notebook shared in the context is to demonstrate the effectiveness of using       │
│ Retrieval-Augmented Generation (RAG) to cite sources and prevent hallucination in generating answers.           │
│ Additionally, it aims to compare the responses generated by plain GPT-3.5 with the responses generated by RAG.  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯